In [ ]:
#| default_exp base

`<<<<<<< HEAD`

`=======`

In [ ]:
from nbdev import *
import requests

`>>>>>>> c2fb413 (update fastai and fastcore)`

import requests

`<<<<<<< HEAD`

In [ ]:
#| export
import requests
from nbdev.showdoc import *
from importlib import reload

`=======`

In [ ]:
#| export
class ISTAT:
    """Base class that provides useful functions to communicate with ISTAT API"""

    def __init__(self):
        self.base_url = "http://sdmx.istat.it/SDMXWS/rest"
        self.agencyID = "IT1"

    def _request(self, path, **kwargs):
        """Make a request to ISTAT API given a 'path'"""
        url = "/".join([self.base_url, path])

        if "headers" in kwargs.keys():
            response = requests.get(url, headers=kwargs["headers"])
        else:
            response = requests.get(url)

        return response

`>>>>>>> c2fb413 (update fastai and fastcore)`

In [ ]:
#| export
class ISTAT:
    """Base class that provides useful functions to communicate with ISTAT API"""

    def __init__(self):
        self.base_url = "http://sdmx.istat.it/SDMXWS/rest"
        self.agencyID = "IT1"

    def _request(self, path, **kwargs):
        """Make a request to ISTAT API given a 'path'"""
        url = "/".join([self.base_url, path])

        if "headers" in kwargs.keys():
            response = requests.get(url, headers=kwargs["headers"])
        else:
            response = requests.get(url)

        return response

In [ ]:
show_doc(ISTAT._request)

---

### ISTAT._request

>      ISTAT._request (path, **kwargs)

Make a request to ISTAT API given a 'path'

The 'ISTAT' class already implements the 'base_url', therefore 'path' only needs to include the other parameters of the request. Here is an example of a succesful request:

In [ ]:
istat = ISTAT()
path = "dataflow/IT1"
test_response = istat._request(path=path)
test_response

<Response [200]>